In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'tpu-getting-started:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F21154%2F1243559%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240207%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240207T130447Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D589abaeb64d60d135b1d18072cf886233a2e511791eddbcc6c7db90f2f7701deaf77a53037ed85a79b3ab93833d4c28c9ecca22ebc1f5f6de6d3c6ff1b710897d64c9d97dfbff98689b107f57fd5c603dd7c83fc3a480879079014b66e0e3199f7c4bb019a0a30328972d610081c6c3ced1f2ae61184c63fc7da64b7900d6cfa17d4cb783e079c22f4a9ddfacc357badc82e279ce568f754641c82e32411f16e4b8f32d8638aa5d1b1e48f229d8c2ba27ea3587db3083a69ff6fbf573970646dca1c0585bf9777dc409ac68338bd166f3b312f2ef14c3fa92172fa222357c53d314e6f6bc184971326c914225e4fd7bdbf4e451f30683ef0d99e5b1b5199a97a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Подготовка

## Импорты

In [ ]:
import random
import tensorflow as tf
# последовательная модель (стек слоев)
from tensorflow.keras.models import Sequential, Model
# полносвязный слой и слой выпрямляющий матрицу в вектор
from tensorflow.keras.layers import Dense, Flatten, Input
# слой выключения нейронов и слой нормализации выходных данных (нормализует данные в пределах текущей выборки)
from tensorflow.keras.layers import Dropout, BatchNormalization, SpatialDropout2D, GaussianDropout
# слои свертки и подвыборки
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
# работа с обратной связью от обучающейся нейронной сети
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# вспомогательные инструменты
from tensorflow.keras import utils
from tensorflow.keras.regularizers import *
import numpy as np
import os
from tensorflow.random import set_seed
def seed_everything(seed):
    np.random.seed(seed)
    set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 42
seed_everything(seed)

# работа с изображениями
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

#  библиотека для работы с наборами данных на Kaggle
from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
%matplotlib inline
print("Tensorflow version " + tf.__version__)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Обнаружение оборудования, возврат соответствующей стратегии распространения: TPU, GPU, CPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Обнаружение TPU. Параметры среды не требуются, если задана переменная среды TPU_NAME. На Kaggle это всегда так.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # стратегия распространения по умолчанию в Tensorflow. Работает на CPU и одном GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

## Данные

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path()
GCS_DS_PATH#получаем путь к наборам данных

In [ ]:
IMAGE_SIZE = [192, 192] # при таком размере графическому процессору не хватит памяти. Используйте TPU
EPOCHS = 25
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

NUM_TRAINING_IMAGES = 12753
NUM_TEST_IMAGES = 7382
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE # находим количество шагов за эпоху

SEED = 2020

## Функции

In [ ]:
def decode_image(image_data):
    """Декодирует изображение в vyjujvthye. vfnhbwe (тензор)
    Нормализует данные и преобразовывает изображения к указанному размеру"""
    image = tf.image.decode_jpeg(image_data, channels=3) # Декодирование изображения в формате JPEG в тензор uint8.
    image = tf.cast(image, tf.float32) / 255.0  # преобразовать изображение в плавающее в диапазоне [0, 1]
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # явный размер, необходимый для TPU
#     image = tf.keras.applications.inception_resnet_v2.preprocess_input(image)
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string означает байтовую строку
        "class": tf.io.FixedLenFeature([], tf.int64),  # [] означает отдельный элемент
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT) # парсим отдельный пример в указанном формате
    image = decode_image(example['image']) # преобразуем изображение к нужному нам формату
    label = tf.cast(example['class'], tf.int32)
    return image, label # возвращает набор данных пар (изображение, метка)

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string означает байтовую строку
        "id": tf.io.FixedLenFeature([], tf.string),  # [] означает отдельный элемент
        # класс отсутствует, задача этого конкурса - предсказать классы цветов для тестового набора данных
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image']) # преобразуем изображение к нужному нам формату
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    """Читает из TFRecords. Для оптимальной производительности одновременное чтение из нескольких
    файлов без учета порядка данных. Порядок не имеет значения, поскольку мы все равно будем перетасовывать данные"""

    ignore_order = tf.data.Options() # Представляет параметры для tf.data.Dataset.
    if not ordered:
        ignore_order.experimental_deterministic = False # отключить порядок, увеличить скорость

    dataset = tf.data.TFRecordDataset(filenames) # автоматически чередует чтение из нескольких файлов
    dataset = dataset.with_options(ignore_order) # использует данные сразу после их поступления, а не в исходном порядке
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord)
    # возвращает набор данных пар (изображение, метка), если метка = Истина, или пар (изображение, идентификатор), если метка = Ложь
    return dataset

def get_training_dataset():
    dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-192x192/train/*.tfrec'), labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # набор обучающих данных должен повторяться в течение нескольких эпох
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

def get_validation_dataset():
    dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-192x192/val/*.tfrec'), labeled=True, ordered=False)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache() # кешируем набор
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-192x192/test/*.tfrec'), labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

## Добавим аугументацию

In [ ]:
def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    flag = random.randint(1,3)
    # coef_1 = random.randint(70, 90) * 0.01
    # coef_2 = random.randint(70, 90) * 0.01
    if flag == 1:
        image = tf.image.random_flip_left_right(image, seed=SEED)
    elif flag == 2:
        image = tf.image.random_flip_up_down(image, seed=SEED)
    # else:
        # image = tf.image.random_crop(image, [int(IMAGE_SIZE[0]*coef_1), int(IMAGE_SIZE[0]*coef_2), 3],seed=SEED, )
    return image, label

## Печать резлуьтатов

In [ ]:
def print_score(history):

  plt.plot(history.history['sparse_categorical_accuracy'],
          label='Оценка точности на обучающем наборе')
  plt.plot(history.history['val_sparse_categorical_accuracy'],
          label='Оценка точности на проверочном наборе')
  plt.xlabel('Эпоха обучения')
  plt.ylabel('Оценка точности')
  plt.legend()
  plt.show()

def print_loss(history):

  plt.plot(history.history['loss'],
         label='Оценка потерь на обучающем наборе')
  plt.plot(history.history['val_loss'],
          label='Оценка потерь на проверочном наборе')
  plt.xlabel('Эпоха обучения')
  plt.ylabel('Оценка потерь')
  plt.legend()
  plt.show()


## Датасеты

In [ ]:
training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()

# Модель

In [ ]:
# def get_model():

#     # Создаем последовательную модель
#     model = Sequential()

#     ## Первый сверточный блок
#     # Первый сверточный слой
#     model.add(Conv2D(64, (5, 5), input_shape=(*IMAGE_SIZE, 3), activation='relu'))
#     model.add(BatchNormalization())
#     # Второй сверточный слой
#     model.add(Conv2D(64, (5, 5), activation='relu'))
#     model.add(BatchNormalization())
#     # # Первый слой подвыборки
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     # Первый Слой регуляризации Dropout
#     model.add(GaussianDropout(0.25))

#     ## Второй сверточный блок
#     # Четвертый сверточный слой
#     model.add(Conv2D(128, (5, 5), activation='relu'))
#     model.add(BatchNormalization())
#     # Пятый сверточный слой
#     model.add(Conv2D(128, (5, 5), activation='relu'))
#     model.add(BatchNormalization())
#     # Второй слой подвыборки
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     # Второй Слой регуляризации Dropout
#     model.add(GaussianDropout(0.35))

#     ## Третий сверточный блок
#     # Шестой сверточный слой
#     model.add(Conv2D(256, (5, 5), activation='relu'))
#     model.add(BatchNormalization())
#     # Седьмой сверточный слой
#     model.add(Conv2D(256, (5, 5), activation='relu'))
#     model.add(BatchNormalization())
#     # Восьмой сверточный слой
#     model.add(Conv2D(256, (5, 5), activation='relu'))
#     model.add(BatchNormalization())
#     # Третий слой подвыборки
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     # Третий Слой регуляризации Dropout
#     model.add(GaussianDropout(0.45))

#     ## Четвертый сверточный блок
#     # Девятый сверточный слой
#     model.add(Conv2D(512, (5, 5), activation='relu'))
#     model.add(BatchNormalization())
#     # Десятый сверточный слой
#     model.add(Conv2D(512, (5, 5), activation='relu'))
#     model.add(BatchNormalization())
#     # Одиннадцатый сверточный слой
#     model.add(Conv2D(512, (5, 5), activation='relu'))
#     model.add(BatchNormalization())
#     # Четвертый слой подвыборки
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     # Четвертый Слой регуляризации Dropout
#     model.add(GaussianDropout(0.5))

#     ## Полносвязный блок
#     # Слой преобразования данных из 2D представления в плоское
#     model.add(Flatten())
#     # Полносвязный слой для классификации
#     model.add(Dense(1024, activation='relu'))
#     # Четвертый слой нормализации данных
#     model.add(BatchNormalization())
#     # Четвертый Слой регуляризации Dropout
#     model.add(GaussianDropout(0.8))
#     # Выходной полносвязный слой
#     model.add(Dense(104, activation='softmax'))
#     return model


# with strategy.scope():
#     model = get_model()
# model.summary()

In [ ]:
def get_model():
    # Создаем новую последовательную модель
    model = Sequential()

    # Первый сверточный блок
    model.add(Conv2D(128, (2, 2), input_shape=(*IMAGE_SIZE, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(5, 5)))
    model.add(Dropout(0.25))

    # Второй сверточный блок
    model.add(Conv2D(256, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.35))

    # Третий сверточный блок
    model.add(Conv2D(512, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.45))

    # Полносвязный блок
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(104, activation='softmax'))

    return model

with strategy.scope():
    model = get_model()
model.summary()

In [ ]:
callbacks_list = [EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                  ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3),
                  ]

model.compile(
    optimizer='nadam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

historical = model.fit(training_dataset,
          steps_per_epoch=STEPS_PER_EPOCH,
          epochs=EPOCHS,
          callbacks=callbacks_list,
          validation_data=validation_dataset)

In [ ]:
print_score(historical)
print_loss(historical)

# Тест на submission

In [ ]:
test_ds = get_test_dataset(ordered=True)

print('Вычисляем предсказания...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

print('Создание файла submission.csv...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # все в одной партии
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')

![image.png](attachment:8af5ec8c-8370-4db3-ba4a-c27ae41b44e4.png)